# Homework 4: Evaluation Metrics for Classification

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('bank-full.csv',
                   sep=';', 
                   quotechar='"',
                   encoding='utf-8')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [3]:
selected_columns = ['age', 'job', 'marital', 'education',  'balance', 'housing',
                    'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
                    'previous', 'poutcome', 'y']

df_selected = df.copy()[selected_columns]
df_selected.head()

,age,job,marital,education,balance,housing,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,2143,yes,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,29,yes,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,2,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,1506,yes,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,1,no,unknown,5,may,198,1,-1,0,unknown,no


## Split the data

Split your data in train/val/test sets with 60%/20%/20% distribution. Use Scikit-Learn for that (the train_test_split function) and set the seed to 42. Make sure that the target value y is not in your dataframe.

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
df_full_train, df_test = train_test_split(df_selected, test_size = 0.2, random_state = 1)

In [6]:
len(df_full_train), len(df_test)

(36168, 9043)

In [7]:
df_train, df_val = train_test_split(df_full_train, test_size = 0.25, random_state = 1)
len(df_train), len(df_val)

(27126, 9042)

In [8]:
y_train = df_train.y.values
y_val = df_val.y.values
y_test = df_test.y.values

In [10]:
del df_train["y"]
del df_val["y"]
del df_test["y"]

In [11]:
df_full_train.y.value_counts(normalize= True)

y
no     0.882797
yes    0.117203
Name: proportion, dtype: float64

## Question 1: ROC AUC feature importance